https://tomekkorbak.com/2019/10/08/lewis-signaling-games/

In [1]:
import numpy as np
# import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import animation
import imageio

In [2]:
class Sender:

    def __init__(self, n_inputs: int, n_messages: int, eps: float = 1e-6):
        self.n_messages = n_messages
        self.message_weights = np.zeros((n_inputs, n_messages))
        self.message_weights.fill(eps)
        self.last_situation = (0, 0)
        self.probabilidad=np.zeros(10)

    def send_message(self, input: int) -> int:
        probs = np.exp(self.message_weights[input, :])/np.sum(np.exp(self.message_weights[input, :]))
        self.probabilidad=probs
        message = np.random.choice(self.n_messages, p=probs)
        self.last_situation = (input, message)

        print(f"Sender - El estimulo es: {input} y el mensaje es {message}")
        return message

    def learn_from_feedback(self, reward: int) -> None:
        self.message_weights[self.last_situation] += reward
        # print(f"la recompenza de sender es: {self.message_weights}")


In [3]:
class Receiver:

    def __init__(self, n_messages: int, n_actions: int, eps: float = 1e-6):
        self.n_actions = n_actions
        self.action_weights = np.ndarray((n_messages, n_actions))
        self.action_weights.fill(eps)
        self.last_situation	 = (0, 0)
        self.probabilidad=np.zeros(10)

    def act(self, message: int) -> int:
        probs = np.exp(self.action_weights[message, :])/np.sum(np.exp(self.action_weights[message, :]))
        #print(type(probs))
        self.probabilidad=probs
        action = np.random.choice(self.n_actions, p=probs)
        self.last_situation = (message, action)
        print(f"Receiver - El mensaje es: {message} y el estimulo es {action}")
        return action

    def learn_from_feedback(self, reward: int) -> None:
        self.action_weights[self.last_situation] += reward
        # print(f"la recompenza de sender es: {self.action_weights}")


In [4]:
class World:
    def __init__(self, n_states: int, seed: int = 1701):
        self.n_states = n_states
        self.state = 0
        self.rng = np.random.RandomState(seed)

    def emit_state(self) -> int:
        self.state = self.rng.randint(self.n_states)
        print(f"World - El estado es: {self.state}")
        return self.state

    def evaluate_action(self, action: int) -> int:
        return 1 if action == self.state else -1


In [9]:
import numpy as np
import matplotlib.pyplot as plt

sender, receiver = Sender(10, 10), Receiver(10, 10)
world = World(10)
past_rewards = 0
matrices = []

for epoch in range(5000):
    world_state = world.emit_state()
    message = sender.send_message(world_state)
    action = receiver.act(message)
    reward = world.evaluate_action(action)
    receiver.learn_from_feedback(reward)
    sender.learn_from_feedback(reward)

    past_rewards += reward

    if epoch % 25 == 0:
        plt.figure(figsize=(6,6))

        # **Heatmap para Receiver**
        data = np.exp(receiver.action_weights)
        normalized_data = data / data.sum(axis=0)
        #print(data)
        plt.imshow(normalized_data, cmap="viridis", aspect="equal")
        plt.colorbar(label="Normalized Weights")

        # Agregar valores dentro de las celdas
        for i in range(normalized_data.shape[0]):
            for j in range(normalized_data.shape[1]):
                plt.text(j, i, f"{normalized_data[i, j]:.1f}", ha="center", va="center", color="white")

        plt.xlabel("Messages")
        plt.ylabel("Actions")
        plt.title(f"Receiver's weights, rollout {epoch}")
        plt.savefig(f"receiver_{epoch}.png")
        plt.clf()


        plt.figure()
        plt.bar(range(len(receiver.probabilidad)), receiver.probabilidad, color='skyblue')
        plt.ylim(0, 1)
        plt.title(f"Paso {epoch}")
        plt.savefig(f"/content/sample_data/receiver_{epoch:03d}.png")
        plt.clf()

        # **Heatmap para Sender**
        data = np.exp(sender.message_weights)
        normalized_data = data / data.sum(axis=0)

        plt.imshow(normalized_data, cmap="viridis", aspect="equal")
        plt.colorbar(label="Normalized Weights")

        for i in range(normalized_data.shape[0]):
            for j in range(normalized_data.shape[1]):
                plt.text(j, i, f"{normalized_data[i, j]:.1f}", ha="center", va="center", color="white")

        plt.xlabel("World states")
        plt.ylabel("Messages")
        plt.title(f"Sender's weights, rollout {epoch}")
        plt.savefig(f"sender_{epoch}.png")
        plt.clf()

        plt.figure()
        plt.bar(range(len(sender.probabilidad)), sender.probabilidad, color='skyblue')
        plt.ylim(0, 1)
        plt.title(f"Paso {epoch}")
        plt.savefig(f"/content/sample_data/sender_{epoch:03d}.png")
        plt.clf()


    if epoch % 100 == 0:
        print(f"Epoch {epoch}, last 100 epochs reward: {past_rewards / 100}")
        print(world_state, message, action, reward)
        past_rewards = 0

print("Observation to message mapping:")
print(sender.message_weights.argmax(1))
print("Message to action mapping:")
print(receiver.action_weights.argmax(1))

World - El estado es: 4
Sender - El estimulo es: 4 y el mensaje es 7
Receiver - El mensaje es: 7 y el estimulo es 1
Epoch 0, last 100 epochs reward: -0.01
4 7 1 -1
World - El estado es: 3
Sender - El estimulo es: 3 y el mensaje es 1
Receiver - El mensaje es: 1 y el estimulo es 2
World - El estado es: 3
Sender - El estimulo es: 3 y el mensaje es 4
Receiver - El mensaje es: 4 y el estimulo es 6
World - El estado es: 5
Sender - El estimulo es: 5 y el mensaje es 7
Receiver - El mensaje es: 7 y el estimulo es 8
World - El estado es: 0
Sender - El estimulo es: 0 y el mensaje es 6
Receiver - El mensaje es: 6 y el estimulo es 4
World - El estado es: 0
Sender - El estimulo es: 0 y el mensaje es 2
Receiver - El mensaje es: 2 y el estimulo es 9
World - El estado es: 4
Sender - El estimulo es: 4 y el mensaje es 6
Receiver - El mensaje es: 6 y el estimulo es 1
World - El estado es: 2
Sender - El estimulo es: 2 y el mensaje es 9
Receiver - El mensaje es: 9 y el estimulo es 4
World - El estado es: 8


<ipython-input-9-5d73a9b3c3be>:65: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


Se han truncado las últimas 5000 líneas del flujo de salida.
Sender - El estimulo es: 7 y el mensaje es 1
Receiver - El mensaje es: 1 y el estimulo es 7
World - El estado es: 3
Sender - El estimulo es: 3 y el mensaje es 2
Receiver - El mensaje es: 2 y el estimulo es 3
World - El estado es: 1
Sender - El estimulo es: 1 y el mensaje es 0
Receiver - El mensaje es: 0 y el estimulo es 1
World - El estado es: 4
Sender - El estimulo es: 4 y el mensaje es 7
Receiver - El mensaje es: 7 y el estimulo es 4
World - El estado es: 5
Sender - El estimulo es: 5 y el mensaje es 5
Receiver - El mensaje es: 5 y el estimulo es 5
World - El estado es: 5
Sender - El estimulo es: 5 y el mensaje es 5
Receiver - El mensaje es: 5 y el estimulo es 5
World - El estado es: 2
Sender - El estimulo es: 2 y el mensaje es 3
Receiver - El mensaje es: 3 y el estimulo es 2
World - El estado es: 0
Sender - El estimulo es: 0 y el mensaje es 6
Receiver - El mensaje es: 6 y el estimulo es 0
World - El estado es: 7
Sender - El

<Figure size 600x600 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 600x600 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [10]:
def make_gif(filename_base):
    images = []
    for filename in [f'{filename_base}_{i}.png' for i in range(5000) if i % 25== 0]:
        images.append(imageio.imread(filename))
    imageio.mimsave(f'/content/sample_data/{filename_base}.gif', images)

make_gif('sender')
make_gif('receiver')

<ipython-input-10-ecb4d89dea34>:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))


In [7]:
import imageio

# Crear el GIF
with imageio.get_writer('grafico_barrasR.gif', mode='I', duration=0.5) as writer:
    for i in range(0, 3000, 25):
            filename = f'/content/sample_data/receiver_{i:03d}.png'
            image = imageio.imread(filename)
            writer.append_data(image)


<ipython-input-7-6f39bde9e2f7>:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [8]:
import imageio

# Crear el GIF
with imageio.get_writer('grafico_barrasS.gif', mode='I', duration=0.5) as writer:
    for i in range(0, 3000, 25):
                filename = f'/content/sample_data/sender_{i:03d}.png'
                image = imageio.imread(filename)
                writer.append_data(image)


<ipython-input-8-7bdbf0b4e20b>:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
